# Teaching Matrix
Richard M. Murray, 18 Jan 2021

This notebook generates the data needed for the BBE "teaching matrix" by reading in information provided by the registrar.  The data are parsed into data entries via custom parsing routes for each spreadsheet.

In [ ]:
import pandas as pd
import numpy as np
import re
import bbedata as bbe

In [ ]:
# Create a dataframe to store the result
bbe_df = pd.DataFrame(columns=[
    "Course", "Title", "Option", "Instructor", "Units", "AY", "Term", "Time", "Size"])

In [ ]:
%load_ext autoreload
%autoreload 2
bbe.__version__

### List of BBE options

List of BBE options that we should be tracking

In [ ]:
bbe_options = ["BE", "Bi", "BMB", "CNS", "NB"]
bbe_pattern = re.compile(r".*(BE|Bi|BMB|CNS|NB)+.*")

### Teaching

Go through the various course data and collect up the results.

In [ ]:
# Read data from teaching assignments
courses = bbe.read_regis_data("All courses past 5 years 210120.xlsx", last_name_only=True)
years = courses['AY'].unique()
courses.info()

In [ ]:
courses.head(50)

In [ ]:
# Create a new data frame to store things by courses
course_df = pd.DataFrame(
    columns=
    ["Course", "Title", "Option req'd", "Units", "2020-21 term", "2020-21 instructor", 
     "Instructor(s)", "Term"] +
    ["%s %s" % (year, prop) for year in years[::-1]
     for prop in ["Instructor(s)", "enroll"]]                     
)

for course in sorted(courses['Course'].unique()):
    this_course = courses[courses['Course'] == course]
    
    # Skip research courses
    if np.all(this_course['Type'].unique() == 'Research'):
        continue
        
    # Create dictionary of properties
    course_props = {'Course': course, 'Title': "; ".join(this_course['Title'].unique())}
    
    # Get the list of instructors
    course_props['Instructor(s)'] = "; ".join(sorted(this_course["Instructors"].unique()))
    
    # Get the terms it is taught
    course_props['Term'] = ", ".join(sorted(this_course["Term"].unique()))
    
    # Get the instructors and enrollments by year
    for year in years:
        this_year = this_course[this_course['AY'] == year]
        if this_year["Instructors"].size > 0:
            course_props["%s Instructor(s)" % year] = ", ".join(this_year["Instructors"].unique())
            course_props["%s enroll" % year] = sum(this_year["Enrollment"])
        
    course_df = course_df.append(course_props, ignore_index=True)

In [ ]:
course_df.head(10)

In [ ]:
# Write the data frame
course_df.drop_duplicates(inplace=True)
course_df.to_excel("BBE Teaching Matrix Template.xlsx", "Courses", index=False)

## Update teaching matrix

This section of the code reads in an existing teaching matrix (in case it has been modified) and updates with responses from the teaching survey.

In [ ]:
# Read the teaching matrix
course_df = pd.read_excel("BBE Teaching Matrix.xlsx", header=1)
course_df

In [ ]:
# Read responses from the survey
survey_df = bbe.load_teaching_survey("BBE Teaching Interest Survey, 2021-22 (Responses).xlsx")
survey_df

In [ ]:
# Update the teaching matrix
bbe.update_teaching_matrix(course_df, survey_df)
course_df.iloc[13]

In [ ]:
# Write out the new teaching matrix
course_df.to_excel("BBE Teaching Matrix Updated.xlsx", "Courses")

In [ ]:
course_df

In [ ]:
survey_df